<a href="https://www.kaggle.com/code/bartekmietlicki/recipes-rag?scriptVersionId=259534983" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

___
___
## **0 - Introduction**
___
___


### The goal of this project is to build a RAG system that recommends recipes based on user queries. The system uses over 62,000 recipes from allrecipes.com.

---

### 0.1 - Project Division

The project is divided into 2 stages:

**I) Building the components of the RAG system**
- training a lightweight summarization model (BART + LoRA) on ~200 hand-crafted examples,
- generating summaries for the full 62k dataset,
- computing embeddings of these summaries with all-mpnet-base-v2,
- building a FAISS index.

**II) Building and invoking the RAG system**
- vectorizing the user query and retrieving top-k matches from FAISS,
- composing the response: recipe name, summary, key ingredients,
- exposing a function for interactive querying.

---

### 0.2 - Usage Instruction

Stage I is time-consuming and requires a GPU. You can either run the full pipeline or use RAG-only with prebuilt artifacts.  
- **Quick start (RAG-only):** set `RAG_ONLY = True`.  
- **Full pipeline:** set `RAG_ONLY = False`.

If `RAG_ONLY = True`, the notebook will use artifacts stored in the Hugging Face repo: **bartekmietlicki/recipes-rag**.

If `RAG_ONLY = False` it is recommended to use a GPU-based environment.

---

### 0.3 - GROQ Key

For the generative component (RAG) to work, a **`GROQ_API_KEY`** is required. You can obtain the key in Groq Cloud: https://console.groq.com/keys.

In your chosen environment (Colab / Kaggle), add a secret named `GROQ_API_KEY` in the Secrets panel. The notebook picks it up automatically.


In [1]:
# RAG Only (Selecting the scope of notebook operation)
RAG_ONLY = True

___
___
## **1 - Setting the Environment & Data Importing**
___
___


### 1.1 - Setting the Environment
___



In [2]:
# Installs & imports
import os, sys, subprocess, contextlib
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"        
os.environ["GLOG_minloglevel"] = "3"            
os.environ["ABSL_LOGGING_MIN_LOG_LEVEL"] = "3" 

subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q", "--no-deps",
     "faiss-cpu==1.8.0.post1", "groq==0.11.0", "sentence-transformers==2.6.1", 
     "httpx==0.27.2", "fastparquet==2024.5.0"],
    check=True
)

@contextlib.contextmanager
def silence_stderr():
    saved_fd = os.dup(2)
    try:
        with open(os.devnull, "w") as dn:
            os.dup2(dn.fileno(), 2)
            yield
    finally:
        os.dup2(saved_fd, 2)
        os.close(saved_fd)

with silence_stderr():
    import json, re, ast, random, zipfile, tempfile
    import numpy as np
    import pandas as pd
    import torch
    import requests
    import gdown
    import faiss

    from datasets import Dataset, DatasetDict
    from transformers import (
        AutoTokenizer, AutoModelForSeq2SeqLM,
        DataCollatorForSeq2Seq,
        Seq2SeqTrainer, Seq2SeqTrainingArguments,
        EarlyStoppingCallback,
    )
    from peft import LoraConfig, get_peft_model
    from huggingface_hub import HfApi, hf_hub_download
    from sentence_transformers import SentenceTransformer
    from groq import Groq

    try:
        from google.colab import userdata
    except Exception:
        userdata = None

# Display
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 200)

# Reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); _ = torch.manual_seed(SEED)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.8 MB/s eta 0:00:00


In [3]:
# Device policy (T4 fp16 if CUDA, else CPU)
HAS_CUDA = torch.cuda.is_available()
USE_FP16 = bool(HAS_CUDA)
PIN_MEMORY = bool(HAS_CUDA)
print(f"Device: {'CUDA' if HAS_CUDA else 'CPU'} | fp16={USE_FP16}")

Device: CPU | fp16=False


### 1.2 - API Keys & Tokens
___

1.2.1 - Groq

In [4]:
def _load_groq_api_key():
    k = os.environ.get("GROQ_API_KEY")
    if k:
        return k

    # 2) Kaggle Secrets
    try:
        from kaggle_secrets import UserSecretsClient
        k = UserSecretsClient().get_secret("GROQ_API_KEY")
        if k:
            return k
    except Exception:
        pass

    # 3) Colab Secrets
    try:
        from google.colab import userdata
        k = userdata.get("GROQ_API_KEY")
        if k:
            return k
    except Exception:
        pass

    # Else:
    raise RuntimeError(
        "GROQ_API_KEY not found. Set it as:\n"
        "- Kaggle: Add-ons → Secrets → name 'GROQ_API_KEY'\n"
        "- Colab:  Tools → Secrets → name 'GROQ_API_KEY'\n"
        "- Or export as environment variable before running the notebook."
    )

GROQ_API_KEY = _load_groq_api_key()
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [5]:
hdrs = {"Authorization": f"Bearer {os.environ['GROQ_API_KEY']}"}
r = requests.get("https://api.groq.com/openai/v1/models", headers=hdrs, timeout=20)
r.raise_for_status()
models = sorted(m["id"] for m in r.json().get("data", []))

print(f"Available models on Groq ({len(models)}):")
for m in models:
    print(" -", m)

Available models on Groq (20):
 - allam-2-7b
 - compound-beta
 - compound-beta-mini
 - deepseek-r1-distill-llama-70b
 - gemma2-9b-it
 - llama-3.1-8b-instant
 - llama-3.3-70b-versatile
 - meta-llama/llama-4-maverick-17b-128e-instruct
 - meta-llama/llama-4-scout-17b-16e-instruct
 - meta-llama/llama-guard-4-12b
 - meta-llama/llama-prompt-guard-2-22m
 - meta-llama/llama-prompt-guard-2-86m
 - moonshotai/kimi-k2-instruct
 - openai/gpt-oss-120b
 - openai/gpt-oss-20b
 - playai-tts
 - playai-tts-arabic
 - qwen/qwen3-32b
 - whisper-large-v3
 - whisper-large-v3-turbo


### 1.3 - Data Importing
___



1.3.1 - Hugging Face

In [6]:
HF_REPO = "bartekmietlicki/recipes-rag"

PATH_RECIPES   = "data/recipes.parquet"
PATH_FT200_ZIP = "data/recipes_ft200.zip"
PATH_ART_ZIP   = "artifacts.zip"

def read_parquet_from_zip(zip_path):
    with zipfile.ZipFile(zip_path, "r") as zf:
        names = [n for n in zf.namelist() if n.lower().endswith(".parquet")]
        assert names, "No .parquet file found inside the zip."
        with zf.open(names[0]) as f:
            return pd.read_parquet(f, engine="fastparquet")

# Load input datasets
recipes_p = hf_hub_download(HF_REPO, PATH_RECIPES, repo_type="dataset")
recipes_df = pd.read_parquet(recipes_p, engine="fastparquet")

ft200_zip_p = hf_hub_download(HF_REPO, PATH_FT200_ZIP, repo_type="dataset")
ft200_df = read_parquet_from_zip(ft200_zip_p)

print(f"recipes_df: {recipes_df.shape} | cols={list(recipes_df.columns)}")
print(f"ft200_df  : {ft200_df.shape} | cols={list(ft200_df.columns)}")
display(recipes_df.head(2))

# Artifacts (Summaries & Embeddings & Index & Ids)
embeddings = None
faiss_index = None
ids = None
recipes_sum_df = None

if RAG_ONLY:
    tmpdir = tempfile.mkdtemp(prefix="hf_artifacts_")
    art_zip_p = hf_hub_download(HF_REPO, PATH_ART_ZIP, repo_type="dataset")

    with zipfile.ZipFile(art_zip_p, "r") as zf:
        zf.extractall(tmpdir)

    emb_path = os.path.join(tmpdir, "artifacts", "embeddings.npy")
    idx_path = os.path.join(tmpdir, "artifacts", "index.faiss")
    ids_path = os.path.join(tmpdir, "artifacts", "id_list.json")
    sum_path = os.path.join(tmpdir, "artifacts", "recipes_sum.parquet")

    # Load artifacts
    embeddings  = np.load(emb_path).astype(np.float32)
    faiss_index = faiss.read_index(idx_path)
    with open(ids_path) as f:
        ids = json.load(f)
    if os.path.exists(sum_path):
        recipes_sum_df = pd.read_parquet(sum_path, engine="fastparquet")

    print(f"\nArtifacts loaded from {PATH_ART_ZIP}")
    print(f"embeddings: {embeddings.shape}, index.ntotal={getattr(faiss_index,'ntotal',None)}, ids={len(ids)}")
    if recipes_sum_df is not None:
        print(f"recipes_sum_df: {recipes_sum_df.shape}")
        display(recipes_sum_df.head(2))

data/recipes.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

data/recipes_ft200.zip:   0%|          | 0.00/75.6k [00:00<?, ?B/s]

recipes_df: (61983, 4) | cols=['name', 'description', 'directions', 'ingredients']
ft200_df  : (200, 6) | cols=['name', 'description', 'directions', 'ingredients', 'summary', 'hashtags']


,name,description,directions,ingredients
0,Deconstructed Screwdriver (The Raw Egg),Frozen ball of orange juice bathed in vodka. I get a lot of comments that it looks like a raw egg!,"Fill a food-safe silicon-based round ice mold will orange juice; freeze until solid, 2 to 3 hours. Carefully open the ice mold and place the orange juice ball into a martini glass; add vodka.","[""orange juice"", ""jiggers vodka""]"
1,Kettle Corn,"Kettle corn is an old-fashioned, county fair treat. Your family will never want plain popcorn again! If you use white sugar, it will taste like popcorn balls; if you use brown sugar, it will taste...",Heat vegetable oil in a large pot over medium heat. Stir in popcorn kernels and sugar. Cover and shake the pot constantly to prevent sugar from burning. When popping has slowed to once every 2 to ...,"[""vegetable oil"", ""white sugar"", ""popcorn kernels""]"


artifacts.zip:   0%|          | 0.00/382M [00:00<?, ?B/s]


Artifacts loaded from artifacts.zip
embeddings: (61983, 768), index.ntotal=61983, ids=61983
recipes_sum_df: (61983, 5)


,name,description,directions,ingredients,summary_gen
0,Deconstructed Screwdriver (The Raw Egg),Frozen ball of orange juice bathed in vodka. I get a lot of comments that it looks like a raw egg!,"Fill a food-safe silicon-based round ice mold will orange juice; freeze until solid, 2 to 3 hours. Carefully open the ice mold and place the orange juice ball into a martini glass; add vodka.","[""orange juice"", ""jiggers vodka""]","Deconstructed orange juice with vodka, jiggers vodka, and a martini glass."
1,Kettle Corn,"Kettle corn is an old-fashioned, county fair treat. Your family will never want plain popcorn again! If you use white sugar, it will taste like popcorn balls; if you use brown sugar, it will taste...",Heat vegetable oil in a large pot over medium heat. Stir in popcorn kernels and sugar. Cover and shake the pot constantly to prevent sugar from burning. When popping has slowed to once every 2 to ...,"[""vegetable oil"", ""white sugar"", ""popcorn kernels""]","Classic, old-fashioned Kettle corn made with vegetable oil, white sugar, and brown sugar."


___
___
## **2 - Fine Tuning of Summary Model (BART + LoRA)**
___
___


###2.1 Prepering Input For Training
____

In [7]:
if RAG_ONLY:
    print("RAG_ONLY=True → skipping 2.1")
else:
    import pandas as pd

    df = ft200_df.fillna("")
    input_text = (
        "Name: "        + df["name"].astype(str).str.strip() + "\n" +
        "Description: " + df["description"].astype(str).str.strip() + "\n" +
        "Ingredients: " + df["ingredients"].astype(str).str.strip() + "\n" +
        "Directions: "  + df["directions"].astype(str).str.strip()
    )
    output_text = df["summary"].astype(str).str.strip()

    train_pairs = pd.DataFrame({"input": input_text, "output": output_text})
    print(f"train_pairs: {train_pairs.shape}")
    display(train_pairs.head(3))


RAG_ONLY=True → skipping 2.1


###2.2 Model Fine Tuning
___

In [8]:
if RAG_ONLY:
    print("RAG_ONLY=True → skipping 2.2")
else:
    import torch
    from datasets import Dataset, DatasetDict
    from transformers import (
        AutoTokenizer, AutoModelForSeq2SeqLM,
        DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
    )
    from peft import LoraConfig, get_peft_model
    from transformers.trainer_callback import EarlyStoppingCallback

    assert "train_pairs" in globals(), "Run 2.1 first to create `train_pairs`."

    raw = Dataset.from_pandas(train_pairs[["input","output"]].copy(), preserve_index=False)
    dsplit = raw.train_test_split(test_size=0.2, seed=42)
    dset = DatasetDict(train=dsplit["train"], validation=dsplit["test"])

    BASE_MODEL = "facebook/bart-base"
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)

    def preprocess(ex):
        enc = tokenizer(ex["input"], max_length=896, truncation=True)
        tgt = tokenizer(text_target=ex["output"], max_length=128, truncation=True)
        enc["labels"] = tgt["input_ids"]
        return enc

    tok = dset.map(preprocess, batched=True, remove_columns=dset["train"].column_names)
    collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

    base = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL)
    lora = LoraConfig(
        r=64, lora_alpha=128, lora_dropout=0.05,
        target_modules=["q_proj","k_proj","v_proj","out_proj","fc1","fc2"],
        bias="none", task_type="SEQ_2_SEQ_LM", modules_to_save=["lm_head"]
    )
    model = get_peft_model(base, lora)

    args = Seq2SeqTrainingArguments(
        output_dir="/content/model_bart_lora",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        learning_rate=2e-4,
        num_train_epochs=4,
        weight_decay=0.01,
        warmup_ratio=0.05,
        lr_scheduler_type="cosine",
        fp16=torch.cuda.is_available(),
        logging_steps=10,
        eval_strategy="epoch",
        save_strategy="no",
        load_best_model_at_end=False,
        predict_with_generate=False,
        report_to=[],
        metric_for_best_model="eval_loss",
        greater_is_better=False
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=tok["train"],
        eval_dataset=tok["validation"],
        data_collator=collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
    )

    trainer.train()
    metrics = trainer.evaluate()
    print("Final eval_loss:", float(metrics.get("eval_loss", float('nan'))))


RAG_ONLY=True → skipping 2.2


___
___
## **3 - Summaries for Recipes**
___
___


###3.1 Prepering Input For Modeling
___

In [9]:
if RAG_ONLY:
    print("RAG_ONLY=True → skipping 3.1")
else:
    df = recipes_df.fillna("")
    recipes_infer = df.copy()
    recipes_infer["input_text"] = (
        "Name: "        + df["name"].astype(str).str.strip() + "\n" +
        "Description: " + df["description"].astype(str).str.strip() + "\n" +
        "Ingredients: " + df["ingredients"].astype(str).str.strip() + "\n" +
        "Directions: "  + df["directions"].astype(str).str.strip()
    )
    display(recipes_infer[["name","input_text"]].head(3))


RAG_ONLY=True → skipping 3.1


###3.2 Modeling Summaries for All Recipes
____

In [10]:
if RAG_ONLY:
    print("RAG_ONLY=True → skipping 3.2")
else:
    import torch
    from tqdm.auto import tqdm

    assert "model" in globals() and "tokenizer" in globals(), "Run 2.2 first (model & tokenizer)."
    assert "recipes_infer" in globals() and "input_text" in recipes_infer, "Run 3.1 first."

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device).eval()

    BATCH_SIZE = 16 if torch.cuda.is_available() else 4
    texts = recipes_infer["input_text"].tolist()
    summaries = []

    for i in tqdm(range(0, len(texts), BATCH_SIZE), desc="Generating"):
        batch = texts[i:i+BATCH_SIZE]
        enc = tokenizer(batch, max_length=896, truncation=True, padding=True, return_tensors="pt")
        enc = {k: v.to(device) for k, v in enc.items()}
        with torch.no_grad():
            out = model.generate(
                **enc, max_new_tokens=128,
                num_beams=4, do_sample=False, early_stopping=True
            )
        summaries.extend(tokenizer.batch_decode(out, skip_special_tokens=True))

    recipes_sum_df = recipes_infer.copy()
    recipes_sum_df["summary_gen"] = summaries
    print(f"Generated summaries: {len(summaries)}")
    display(recipes_sum_df[["name","summary_gen"]].head(3))


RAG_ONLY=True → skipping 3.2


In [11]:
if 'recipes_sum_df' in globals():
    import os
    os.makedirs("/content/artifacts", exist_ok=True)
    recipes_sum_df[["name","description","directions","ingredients","summary_gen"]].to_parquet(
        "/content/artifacts/recipes_sum.parquet", index=False
    )
    print("/content/artifacts/recipes_sum.parquet")
else:
    print("skip: recipes_sum_df not found")


/content/artifacts/recipes_sum.parquet


___
___
## **4 - Embeddings (MPNet-base)**
___
___


In [12]:
if RAG_ONLY:
    print("RAG_ONLY=True → skipping 4 (using prebuilt embeddings).")
else:
    assert "recipes_sum_df" in globals() and "summary_gen" in recipes_sum_df, "Run 3.2 first."
    device = "cuda" if torch.cuda.is_available() else "cpu"

    st_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device=device)
    texts = recipes_sum_df["summary_gen"].astype(str).tolist()

    embeddings = st_model.encode(
        texts,
        batch_size=256 if torch.cuda.is_available() else 64,
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True,
    ).astype(np.float32)

    ids = list(range(len(texts)))
    print("embeddings:", embeddings.shape, "| ids:", len(ids))


RAG_ONLY=True → skipping 4 (using prebuilt embeddings).


In [13]:
if 'embeddings' in globals() and 'ids' in globals():
    import os, json, numpy as np
    os.makedirs("/content/artifacts", exist_ok=True)
    np.save("/content/artifacts/embeddings.npy", embeddings.astype(np.float32))
    with open("/content/artifacts/id_list.json", "w") as f:
        json.dump([int(x) for x in ids], f)
    print("/content/artifacts/embeddings.npy | /content/artifacts/id_list.json")
else:
    print("skip: embeddings/ids not found")


/content/artifacts/embeddings.npy | /content/artifacts/id_list.json


___
___
## **5 - Indexing Recipes (FAISS)**
___
___


In [14]:
if RAG_ONLY:
    print("RAG_ONLY=True → skipping 5 (using prebuilt FAISS).")
else:
    import faiss, math

    assert "embeddings" in globals(), "Run 4 first."
    dim = embeddings.shape[1]

    nlist = max(1, int(2 * math.sqrt(embeddings.shape[0])))

    quantizer = faiss.IndexFlatIP(dim)
    faiss_index = faiss.IndexIVFFlat(quantizer, dim, nlist, faiss.METRIC_INNER_PRODUCT)

    train_size = min(10000, embeddings.shape[0])
    faiss_index.train(embeddings[:train_size])
    faiss_index.add(embeddings)
    faiss_index.nprobe = 16

    print(f"faiss_index.ntotal: {faiss_index.ntotal} | nlist: {nlist} | nprobe: {faiss_index.nprobe}")


RAG_ONLY=True → skipping 5 (using prebuilt FAISS).


In [15]:
if 'faiss_index' in globals():
    os.makedirs("/content/artifacts", exist_ok=True)
    faiss.write_index(faiss_index, "/content/artifacts/index.faiss")
    print("/content/artifacts/index.faiss")
else:
    print("skip: faiss_index not found")


/content/artifacts/index.faiss


___
___
## **6 - Retriever**
___
___


### 6.1 - Retriever Function
___

In [16]:
if 'st_model' not in globals():
    st_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")


def retrieve_similar_recipes(query, top_k=5):
    # Encode + normalize
    query_emb = st_model.encode([query], show_progress_bar=False, convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(query_emb)

    # Search in FAISS index
    distances, indices = faiss_index.search(query_emb, top_k)

    # Map embedding rows → recipe rows (ids list used if exists)
    recipe_rows = [ids[i] for i in indices[0]]

    # Slice recipes dataframe (always use recipes_sum_df in this project)
    similar_recipes = recipes_sum_df.iloc[recipe_rows].copy()
    similar_recipes['similarity_score'] = distances[0]

    return similar_recipes

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 6.2 - Retriever Testing
___

In [17]:
test_queries = [
    "What’s a healthy smoothie recipe with spinach and berries?",
    "Can you recommend a keto-friendly snack recipe?",
    "What’s the best way to grill salmon?",
    "How do I bake a moist chocolate cake?"
]

for q in test_queries:
    print(f"\nQuery: {q}")
    results = retrieve_similar_recipes(q, top_k=3)

    for _, row in results.iterrows():
        print(f"\nRecipe: {row['name']}")
        print(f"Summary: {row['summary'] if 'summary' in row else row['summary_gen']}")
        print(f"Ingredients: {row['ingredients']}")
        print(f"Directions: {row['directions']}")
        print(f"Similarity Score: {row['similarity_score']:.4f}")
    print("-" * 60)


Query: What’s a healthy smoothie recipe with spinach and berries?

Recipe: Healthy Berry and Spinach Smoothie
Summary: Healthy spinach berry smoothie with yogurt, orange juice, and strawberries.
Ingredients: ["frozen berries", "plain yogurt", "orange juice", "fresh spinach", "strawberries"]
Directions: Blend berries, yogurt, orange juice, spinach, and strawberries together in a blender until smooth.
Similarity Score: 0.8816

Recipe: Beet and Berry Smoothie
Summary: A healthy fruit and veggie smoothie made with fresh spinach, yogurt, and fresh blueberries.
Ingredients: ["fresh spinach", "beet", "yogurt", "red raspberries", "frozen blueberries", "cucumber"]
Directions: Blend spinach, beet, yogurt, raspberries, blueberries, ice cubes, and cucumber together in a blender until smooth.
Similarity Score: 0.8372

Recipe: Green Monster - Spinach Smoothie
Summary: Classic spinach smoothie with yogurt, banana, and blueberries, perfect for a variety of vegetables.
Ingredients: ["yogurt", "banana"

___
___
## **7 - Generator (Groq)**
___
___



### 7.1 - Generator
___

In [18]:
# Choose the model here:
GROQ_MODEL_NAME = "llama-3.3-70b-versatile"
# Alternatives:
# GROQ_MODEL_NAME = "deepseek-r1-distill-llama-70b"
# GROQ_MODEL_NAME = "meta-llama/llama-prompt-guard-2-86m"

# Initialize Groq client
groq_client = Groq(api_key=os.environ["GROQ_API_KEY"])

def _build_recipe_context(df):
    blocks = [
        (
            f"Name: {row['name']}\n"
            f"Summary: {row['summary'] if 'summary' in df.columns else row.get('summary_gen', '')}\n"
            f"Ingredients: {row['ingredients']}\n"
            f"Directions: {row['directions']}\n"
        )
        for _, row in df.iterrows()
    ]
    return "\n".join(blocks).strip()

def generate_message(query, top_k=5):
    # Retrieve
    similar_recipes = retrieve_similar_recipes(query, top_k=top_k)
    context = _build_recipe_context(similar_recipes)

    # Compose prompt
    system_text = "You are a helpful assistant."
    human_text = (
        "Based on the following recipes, answer the user's query by selecting the most appropriate recipe "
        "and providing the full details without mentioning that the recipes were provided. "
        "The response should be divided into the following sections:\n"
        "1. Engaging Introduction\n"
        "2. Ingredients (bullet list)\n"
        "3. Directions (step-by-step)\n"
        "4. Summary\n"
        "Ensure that:\n"
        "- You'll start your reply by thanking the user for choosing you to help them choose a recipe.\n"
        "- The introduction is inviting and encourages the user to try the recipe.\n"
        "- In the introduction, try to refer to the user's question and indicate which recipe you have chosen by providing its name.\n"
        "- Ingredients are listed using bullet points wwith some introduction at top\n"
        "- Directions are presented as step-by-step instructions.\n"
        "- Summary provides a brief overview.\n"
        "- Conclusion ends with a phrase like 'Bon Appetit!', 'Enjoy!' or or another phrase encouraging people to try the recipe.\n"
        "- Base the content on the recipe provided and do not invent ingredients or preparation steps.\n"
        "- Don't use headings for 'Engaging Introduction', 'Summary', and 'Conclusion'.\n\n"
        f"Recipes:\n{context}\n\n"
        f"User Query: {query}\n\n"
        "Response:"
    )

    # Call Groq Chat
    resp = groq_client.chat.completions.create(
        model=GROQ_MODEL_NAME,
        temperature=0,
        messages=[
            {"role": "system", "content": system_text},
            {"role": "user",   "content": human_text},
        ],
        max_tokens=1024,
    )

    text = resp.choices[0].message.content or ""
    return text.strip()



### 7.2 - Generator Test
___

In [19]:
test_query = "I need a healthy soup recipe with almonds."
test_out = generate_message(test_query, top_k=5)
print(test_out[:2000])

Thank you for choosing me to help you choose a recipe. I've selected the Elegant Wild Rice Soup, which I think you'll find perfect for your request. This hearty and flavorful soup is not only delicious, but it also features almonds as one of its key ingredients. The combination of wild rice, ham, carrots, and almonds creates a truly satisfying and healthy meal. So, why not give it a try and warm up with a bowl of this nutritious and tasty soup?

To make this soup, you'll need the following ingredients:
* wild rice
* water
* butter
* onion
* all-purpose flour
* chicken broth
* carrots
* almonds

Here's how to prepare the soup:
1. In a pot, bring wild rice and water to a boil.
2. Reduce heat to low, cover, and simmer for 45 minutes.
3. In a separate pot, melt butter over medium heat, and sauté onion until tender.
4. Blend in flour until smooth, and gradually stir in chicken broth.
5. Stirring constantly, bring the mixture to a boil.
6. Reduce heat to low, and mix in cooked rice, ham, car

___
___
## **8 - RAG**
___
___



### 8.2 - Culinary Queries Filter
___

In [20]:
def is_culinary_query(query):
    system_text = "You are a helpful assistant."
    human_text = (
        "Determine whether the following user query contains any words related to food or cooking.\n\n"
        f"User Query: {query}\n\n"
        "Answer with only 'TRUE' or 'FALSE'."
    )

    resp = groq_client.chat.completions.create(
        model=GROQ_MODEL_NAME,
        temperature=0,
        messages=[
            {"role": "system", "content": system_text},
            {"role": "user",   "content": human_text},
        ],
        max_tokens=8,
    )
    txt = (resp.choices[0].message.content or "").strip().upper()
    txt = txt.replace(".", "").replace("!", "").replace(" ", "")
    return txt == "TRUE"


### 8.2 - RAG
___

In [21]:
def RAG(query, top_k=5, use_filter=True, print_query=False):
    if print_query:
        print(f"User Query: {query}")

    if use_filter:
        ok = is_culinary_query(query)
        if not ok:
            print("I'm sorry, I can only assist with culinary-related queries and recipes.")
            return

    response = generate_message(query, top_k=top_k)

    if isinstance(response, str):
        response = response.strip().replace("\u200b", "")
    else:
        response = str(response).strip()

    print(response)

___
___
## **9 - RAG Testing**
___
___


In [22]:
query = "I need a healthy soup recipe with almonds."
RAG(query)

Thank you for choosing me to help you choose a recipe. I've selected the Elegant Wild Rice Soup, which I think you'll find is a perfect fit for your request. This hearty and flavorful soup is not only delicious, but it also features almonds as one of its key ingredients. The combination of wild rice, ham, carrots, and almonds creates a truly satisfying and healthy meal that's sure to become a favorite.

To make this wonderful soup, you'll need the following ingredients:
* wild rice
* water
* butter
* onion
* all-purpose flour
* chicken broth
* carrots
* almonds
* ham

Here's how to prepare the soup:
1. In a pot, bring wild rice and water to a boil.
2. Reduce heat to low, cover, and simmer for 45 minutes.
3. In a separate pot, melt butter over medium heat, and sauté onion until tender.
4. Blend in flour until smooth, and gradually stir in chicken broth.
5. Stirring constantly, bring the mixture to a boil.
6. Reduce heat to low, and mix in cooked rice, ham, carrots, and almonds.
7. Conti

In [23]:
query = "What are the ingredients for a Mediterranean quinoa salad? "
RAG(query)

Thank you for choosing me to help you with your query. I've selected the Mediterranean Quinoa Salad with Shrimp recipe, which I think you'll find perfect for your needs. This refreshing salad combines the flavors of the Mediterranean with the protein-rich shrimp, making it an excellent choice for a light and satisfying meal. The combination of shrimp, cucumber, and feta cheese, marinated in olive oil, creates a delicious and healthy dish that's sure to please.

To make this salad, you'll need the following ingredients:
* Shrimp
* Lemon
* Salt
* Grape tomatoes
* Quinoa
* Cucumber
* Yellow bell pepper
* Green bell pepper
* Kalamata olives
* Red onion
* Feta cheese
* Olive oil
* Balsamic vinegar
* Oregano

Here's how to prepare the salad:
1. Toss shrimp with lemon juice, salt, and black pepper in a bowl; set aside to marinate, about 15 minutes.
2. Mix tomatoes, quinoa, cucumber, yellow bell pepper, green bell pepper, kalamata olives, red onion, and feta cheese together in a large bowl.
3.

In [24]:
query = "cauliflower"
RAG(query)

Thank you for choosing me to help you with a delicious cauliflower recipe. I've selected the 'Cauliflower With a Kick' recipe, which is perfect for summer gatherings and is sure to tantalize your taste buds with its unique blend of flavors. This recipe is a great way to enjoy cauliflower, and its simplicity makes it an excellent choice for anyone looking to try something new. The combination of cauliflower, parsnips, milk, and butter creates a dish that's both comforting and flavorful.

To make this recipe, you'll need the following ingredients:
* Cauliflower
* Parsnip
* Milk
* Butter
* Salt

Here's how to prepare it:
1. Place a steamer insert into a saucepan, and fill with water to just below the bottom of the steamer.
2. Cover, and bring the water to a boil over high heat.
3. Add cauliflower and parsnips, recover, and steam until soft, about 15 minutes.
4. Drain the vegetables.
5. Mash using a hand masher or electric mixer.
6. Gradually add milk and butter and mash to desired consist

In [25]:
query = "How to replace the water pump in my car?"
RAG(query)


I'm sorry, I can only assist with culinary-related queries and recipes.


In [26]:
query = "tomato basil"
RAG(query)

Thank you for choosing me to help you choose a recipe. I think you'll love the Sun-Dried Tomato Pesto, which combines the flavors of fresh basil and tomatoes to create a delicious and versatile sauce. This recipe is perfect for pasta, pizza, and sandwiches, and its unique blend of ingredients will add a new dimension to your favorite dishes. With the sweetness of sun-dried tomatoes and the brightness of fresh basil, this pesto is sure to become a staple in your kitchen.

To make this delicious pesto, you'll need the following ingredients:
* tomatoes
* fresh basil
* parsley
* garlic
* pine nuts
* onion
* balsamic vinegar
* tomato paste
* red wine
* olive oil
* Parmesan cheese
* salt

Here's how to prepare the Sun-Dried Tomato Pesto:
1. Place sun-dried tomatoes in a bowl and cover with warm water for 5 minutes, or until tender.
2. Place sun-dried tomatoes, pine nuts, onion, basil, parsley, and garlic in a food processor or blender; process until well blended.
3. Add crushed tomatoes, vin

In [27]:
RAG("What is the capital of Germany?")

I'm sorry, I can only assist with culinary-related queries and recipes.


___
___
## **10 - Interactive RAG**
___
___



### 10.1 - Interactive RAG Function (Colab only)
___

This section enables an **interactive** mode using `input()`.
> **Note:** Interactive input does **not** work on **Kaggle** (Commit / Save & Run All).  
To use the interactive mode, open the notebook in **Google Colab**:

https://colab.research.google.com/drive/1UcibjTWtYVBQYuAVxIfq2Vs5p8e2doKU#scrollTo=4O5bF3ERxObh

On Kaggle, please use the batch tests from §9 or call `RAG("your query")` directly.


In [28]:
def interactive_RAG(top_k=5, use_filter=True):
    print("Welcome to the Interactive Recipe Assistant!")
    print("Type your culinary query (or 'exit' to quit).")

    while True:
        try:
            query = input("\nEnter your query: ").strip()
        except (EOFError, KeyboardInterrupt):
            print("Thank you. Bye!")
            break

        if not query:
            print("Please enter a non-empty query.")
            continue

        low = query.lower()
        if low in ("exit", "quit", "q"):
            print("Goodbye!")
            break

        try:
            RAG(query, top_k=top_k, use_filter=use_filter, print_query=False)
        except Exception as exc:
            print(f"Error while processing your query: {exc}")


### 10.2 - Sample queries for testing
___

- What is a good pasta recipe?
- I need a soup with tomatoes.
- Can you suggest a vegan salad?
- Healthy breakfast?
- Best chicken recipe?
- How can I make a creamy mushroom risotto for dinner?
- I’m looking for a dessert recipe that uses chocolate and strawberries.
- What’s a quick and easy recipe for a weeknight dinner with chicken?
- Can you suggest a gluten-free snack I can prepare for kids?
- How do I bake a moist chocolate cake with no eggs?
- I need a recipe for a dairy-free lasagna with lots of vegetables.
- How can I prepare a three-course meal with Italian recipes for a family dinner?
- Suggest a low-carb, high-protein lunch recipe for someone on a diet.
- I’m planning a barbecue party. Can you give me recipes for burgers, side dishes, and drinks?
- What are some traditional Indian recipes that are easy to prepare for a beginner?
- What is the capital of Germany?
- How do I fix my washing machine?
- Can you suggest a good movie to watch on a Friday night?
- Who won the last FIFA World Cup?
- Tell me a joke about cats.


### 10.3 - Launch of Interactive RAG

In [29]:
if os.environ.get("KAGGLE_KERNEL_RUN_TYPE") != "Batch":  # runs in edit, skips on Commit
    interactive_RAG(top_k=5, use_filter=True)
else:
    print("Interactive mode is skipped during Kaggle Commit/Run All.")

Interactive mode is skipped during Kaggle Commit/Run All.
